# Push your Model to the Hub

This notebook covers how to use the `hf_hub_push` function to push your model to the Hub. This notebook is based on the documentation provided by HuggingFace in their course, specifically [this chapter](https://huggingface.co/course/chapter4/3?fw=pt).

Documentation on the **Huggingface Hub** can be found [here](https://huggingface.co/docs/hub/main).

_Note: before using this notebook you may want to export your model with the aitextgen function, a helper script is provided in `deploy-as-bot\prep_huggingface.py`_


## getting started

In [1]:
from huggingface_hub import (
    # User management
    login,
    logout,
    notebook_login,
    whoami,

    # Repository creation and management
    create_repo,
    delete_repo,
    update_repo_visibility,

    # And some methods to retrieve/change information about the content
    list_models,
    list_datasets,
    list_metrics,
    list_repo_files,
    upload_file,
    delete_file,
)


In [2]:
notebook_login()


In [3]:
from IPython.display import clear_output
import time
import pprint as pp
pp.pprint(whoami())
time.sleep(30) # clear_output after 30 seconds
clear_output(wait=False)

{'avatarUrl': 'https://aeiljuispo.cloudimg.io/v7/https://s3.amazonaws.com/moonup/production/uploads/1622986287260-noauth.jpeg?w=200&h=200&f=face',
 'email': 'peterszemraj@gmail.com',
 'emailVerified': True,
 'fullname': 'Peter Szemraj',
 'name': 'pszemraj',
 'orgs': [{'apiToken': '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',
           'avatarUrl': 'https://aeiljuispo.cloudimg.io/v7/https://s3.amazonaws.com/moonup/production/uploads/1639063857544-60bccec062080d33f875cd0c.png?w=200&h=200&f=face',
           'email': None,
           'fullname': 'ETHZ Analytics Club',
           'name': 'ethzanalytics',
           'periodEnd': None,
           'plan': 'NO_PLAN',
           'roleInOrg': 'admin',
           'type': 'org'}],
 'periodEnd': None,
 'plan': 'NO_PLAN',
 'type': 'user'}


## Setup your repo

In [ ]:
from pathlib import Path
from huggingface_hub import list_repo_files, Repository
from ipywidgets import interact, interactive, fixed, interact_manual, widgets

my_repo = widgets.Text(
    value='My Hello World Repo',
    placeholder='put repo name here',
    description='Enter the name of the repository: ',
    disabled=False
)


my_repo


In [ ]:
print(f"your repository {my_repo.value} will be created")

In [ ]:
local_path = widgets.Text(
    value=str(Path.cwd()),
    placeholder='Enter the path of the folder to upload',
    description='Filepath: ',
    disabled=False
)
local_path


In [ ]:
print(f"your local path \n {local_path.value} \n will be uploaded")

In [ ]:
org_token = widgets.Password(
    placeholder='Enter your org token',
    description='Org Token:',
    disabled=False
)
print('enter your org token below to publish to your org. otherwise, leave blank.\n')
org_token


In [ ]:
use_org = True if org_token.value else False
print(f'use org? {use_org}, stored org token with length: {len(org_token.value)} chars')

---

## create repo on hub, clone to local

In [ ]:
repo_path = Path.cwd() # clone to local working directory
repo_name = str(my_repo.value)
repo_url = create_repo(name=repo_name) if not use_org else create_repo(repo_type=None, name=repo_name, organization='ethzanalytics', exist_ok=True)
repo = Repository(local_dir=str(repo_path.resolve()), clone_from=repo_url)


## export aitextgen model to repo folder

In [ ]:
import subprocess
nb_dir = Path().cwd()
root = nb_dir.parent
# run deploy-as-bot\prep_huggingface.py with the repo name as an argument
subprocess.run(['python', 'deploy-as-bot/prep_huggingface.py',
                "--model-dir", local_path,
                '--hf-name', repo_name], cwd=root.resolve())

## push to hub - upload like it's hot

In [ ]:
commit_msg = widgets.Text(
    placeholder='Your commit message',
    description='Commit message: ',
    disabled=False
)
commit_msg


In [ ]:
repo.push_to_hub(str(commit_msg.value))